# Serving an Object Detector via TF Serving

Date: May 23, 2020

Author: Thanaphon Chavengsaksongkram

Email: contact@thanaphon.dev


# Introduction and Background

Serving a machine learning prediction can be done by simply running a model against a batch of collected data on a scheduled task. However, in any larger IT operations, it is often a requirement to serve the predictions on a on-demand basis to various part of the IT infrastructure using a common protocal such as REST API.

Furthermore, machine learning models degrade over time (model rotting) and they require updated data to be trained and redeploy to production. This problem introduces many engineering challenges such as versioning, transitioning, deployment and backout, availability, scalability, etc. 

Tensorflow Serving (TF Serving) is a solution designed to tackle many of the engineering tasks. It is an efficient model server written in C++ that can handle high load, serve multiple versions of the model, and deploy model automatically from source control. TF Serving can also be deployed to private onpremise infrastructure as well asl to managed services such as Google Cloud AI Platform with extra benefits such as built-in monitoring and so on. 



# Running TF Serving

There are many ways to install and run TF Serving: using a Docker image, using system’s package manager, or installing from source. It is recommended by the TensorFlow team to use TF Serving docker image, as it is one of the fastest way to get your model to production.  Docker images are generally platform agnostic and can be deployed to various infrastructure. TF Serving Docker images also support easy configuration with one with GPU and one without.

There are two strategies to leverage TF Serving docker images. 
 1. Use the base TF Serving docker image as a generic model server and configure it to serve a specific SavedModel. 
 2. Create a new docker image with a SavedModel model baked into it using TF Serving as a base image. 

The first approach requires less maintenance overhead and it is suitable for most application. The second approach can reduce deployment configuration, which can be useful for deploying a SavedModel to many different platforms.

## Model Format for TF Serving

TF Serving server expects a SavedModel, which represents a version of a model generated by tf.saved_model.save() function. It is stored as a directory containing computation graph and its associated data. SavedModel also provides a CLI tool called saved_model_cli that can be used to make a test prediction to a SavedModel. 

Tensorflow 1.x generally saves a model into a frozen graph format. This is not compatibile with TF serving as it expects a SavedModel. Fortunately, SavedModel is just a wrapper of a frozen graph with additional information such as signatures. Converting a frozen graph to a SavedModel is pretty straightforward. 

# Objective.

1. Deploy the model (locally) using Tensorflow Serving. A little tip: Tensorflow Serving might not be able 
to use the model in its current frozen graph format. Maybe you have to save it in a different format first!
2. Create a Tensorflow Serving docker image
3. Run the docker image and change `image_example.py` to use the external Tensorflow Serving model.
4. (optional) The code and application structure isn't very neat. Feel free to redesign the application structure and
code to create a nicer, more usable client (package)

# 0. Prerequisites


## 0.1 Install the prerequisits given.

In [1]:
!pip install -r requirements.txt

     |████████████████████████████████| 52.1 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 24.5 MB 2.8 MB/s eta 0:00:011
     |████████████████████████████████| 105.8 MB 448 kB/s eta 0:00:011
     |████████████████████████████████| 3.1 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 488 kB 3.9 MB/s eta 0:00:01
ERROR: tensorflow-serving-api 2.1.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.14.0 which is incompatible.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.4
    Uninstalling numpy-1.18.4:
      Successfully uninstalled numpy-1.18.4
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2

## 0.2 Install missing packages

In [3]:
!pip install Image

Processing /Users/thanaphonchavengsaksongkram/Library/Caches/pip/wheels/09/21/3d/d9a06fda40387586027b9963b9558d6b655e0cde968737308f/image-1.5.31-py2.py3-none-any.whl
  Using cached Django-3.0.6-py3-none-any.whl (7.5 MB)
  Using cached Pillow-7.1.2-cp37-cp37m-macosx_10_10_x86_64.whl (2.2 MB)
  Using cached pytz-2020.1-py2.py3-none-any.whl (510 kB)
  Using cached asgiref-3.2.7-py2.py3-none-any.whl (19 kB)
  Using cached sqlparse-0.3.1-py2.py3-none-any.whl (40 kB)
ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/Users/thanaphonchavengsaksongkram/miniconda3/envs/tensorflow/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/Users/thanaphonchavengsaksongkram/miniconda3/envs/tensorflow/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exc

## 0.3 Execute the provided sample script

In [95]:
!python3 image_example.py

Instructions for updating:
Use tf.gfile.GFile.


2020-05-22 23:51:52.790135: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


## 0.4 Imports

In [41]:
import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants
import os

# 1. Exploring the given Frozen Graph Model

In order to convert the frozen graph to a SavedModel, more information regarding the model is required.

## 1.1 Export frozen graph to Tensorboard

The first step is to dump the frozen graph into a directory, then inspect it via Tensorboard.

In [34]:
%load_ext tensorboard
with tf.Session() as sess:
    model_filename ='detector_frozen.pb'
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)
LOGDIR='tensorboard'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)
%tensorboard --logdir tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Instructions for updating:
Use tf.gfile.GFile.


## 1.2 Input and Output layers

According to the Tensorboard, the model contains one input layer and 3 output layers.

### 1.2.1 Input Layer:
- input/input_data

### 1.2.2 Output Layers
- pred_mbbox/concat_2
- pred_sbbox/concat_2
- pred_lbbox/concat_2

## 1.3 Converting Frozen Graph to SavedModel

Export the model into sertis-detector model with a version of 1. The default predict signature definitions will be used for predict API. The input and output tensors are specified based from the information previously obtained.

In [36]:
export_dir = './serving/sertis-detector/1/'
graph_pb = 'detector_frozen.pb'

builder = tf.saved_model.builder.SavedModelBuilder(export_dir)

with tf.gfile.GFile(graph_pb, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

sigs = {}

with tf.Session(graph=tf.Graph()) as sess:
    # name="" is important to ensure we don't get spurious prefixing
    tf.import_graph_def(graph_def, name="")
    g = tf.get_default_graph()
   # print([n.name for n in tf.get_default_graph().as_graph_def().node])
    inp = g.get_tensor_by_name("input/input_data:0")
    pred_mbbox = g.get_tensor_by_name("pred_mbbox/concat_2:0")
    pred_sbbox = g.get_tensor_by_name("pred_sbbox/concat_2:0")
    pred_lbbox = g.get_tensor_by_name("pred_lbbox/concat_2:0")
    sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = \
        tf.saved_model.signature_def_utils.predict_signature_def(
            {"in": inp}, {"out_mbbox": pred_mbbox, "out_sbbox": pred_sbbox, "out_lbbox": pred_lbbox })

    builder.add_meta_graph_and_variables(sess,
                                         [tag_constants.SERVING],
                                         signature_def_map=sigs)

    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./serving/sertis-detector/1/saved_model.pb


## 1.4 Inspect the SavedModel

Inspect the SavedModel using saved_model_cli.

In [38]:
!saved_model_cli show --dir serving/sertis-detector/1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['in'] tensor_info:
      dtype: DT_FLOAT
      shape: unknown_rank
      name: input/input_data:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['out_lbbox'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, -1, 3, 85)
      name: pred_lbbox/concat_2:0
  outputs['out_mbbox'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, -1, 3, 85)
      name: pred_mbbox/concat_2:0
  outputs['out_sbbox'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, -1, 3, 85)
      name: pred_sbbox/concat_2:0
Method name is: tensorflow/serving/predict


The saved model should be ready for deployment.

# 2. Using TF Serving to Serve the Model

Before creating a docker image that serves this model, test if the model can be served by running it on the base TF serving image.

## 2.1 Download Tensorflow Serving Image



In [2]:
!docker pull tensorflow/serving

Using default tag: latest
latest: Pulling from tensorflow/serving

a4a261c9: Pulling fs layer 
20cdee96: Pulling fs layer 
60e1d0de: Pulling fs layer 
7668deea: Pulling fs layer 
b5699598: Pulling fs layer 
8f5dbe31: Pulling fs layer 
011e11a2: Pulling fs layer 
Digest: sha256:ea44bf657f8cff7b07df12361749ea94628185352836bb08065345f5c8284bae
Status: Downloaded newer image for tensorflow/serving:latest
docker.io/tensorflow/serving:latest


### 2.1.1 Verify that the images have been downloaded

In [39]:
!docker images

REPOSITORY           TAG                 IMAGE ID            CREATED             SIZE
jwt-api-test         1.0                 f690ec151b72        5 weeks ago         1.04GB
tensorflow/serving   latest              7c20ddd72597        4 months ago        251MB
python               stretch             b9d77e48a75c        8 months ago        940MB


## 2.2 Set environment variables

Unfortunately, Jupyter notebook does not persist environment variables set by the shell. So python os package will be used instead.

In [42]:
os.environ["MODEL_PATH"] = os.path.join(os.path.sep, os.getcwd(), "serving", "sertis-detector")

In [43]:
!echo $MODEL_PATH

/Users/thanaphonchavengsaksongkram/Projects/ML-Practical/mle-take-home-test/serving/sertis-detector


## 2.3 Start Tensorflow Serving Server 

Start the model server and mount the model to the container file system. 

In [64]:
!docker run -it --rm -p 8500:8500 -p 8501:8501 --name sertis-detector --detach \
        -v "$MODEL_PATH:/models/sertis-detector" \
        -e MODEL_NAME=sertis-detector tensorflow/serving         

docker: Error response from daemon: Conflict. The container name "/sertis-detector" is already in use by container "3f6cc2102b4fe8a0c4d96f4ed2415560a4f08f7cc18f113af283dfbac5839430". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


### 2.3.1 Expplaination of each arguments.

For reference, a short explaination of each parameters are listed here.

#### --detach 
  run the image in the background

#### --name 
  name the container so we can stop or restart it later.
#### -v "$MODEL_PATH:/models/sertis-detector"

Mount the host file system that contains the model to the container file system at the specified path.

#### -e MODEL_NAME=my_mnist_model

Sets the container’s MODEL_NAME environment variable, so TF Serving knows which model to serve. By default, it will look for models in the /models directory, and it will automatically serve the latest version it finds.

#### --rm
Deletes the container when you stop it (no need to clutter your machine with interrupted containers). However, it does not delete the image.

#### -p 8500:8500

Makes the Docker engine forward the host’s TCP port 8500 to the container’s TCP port 8500. By default, TF Serving uses this port to serve the gRPC API.

#### -p 8501:8501
Forwards the host’s TCP port 8501 to the container’s TCP port 8501. By default, TF Serving uses this port to serve the REST API.

### 2.3.2 Verify that the container is running.

In [45]:
!docker ps --all

CONTAINER ID        IMAGE                COMMAND                  CREATED             STATUS                   PORTS                              NAMES
06dfb17894b5        tensorflow/serving   "/usr/bin/tf_serving…"   2 hours ago         Up 2 hours               0.0.0.0:8500-8501->8500-8501/tcp   sertis-object-detector
cbf5a53282d3        jwt-api-test:1.0     "gunicorn -b :8080 m…"   5 weeks ago         Exited (0) 5 weeks ago                                      my-app


## 2.4 Testing the model

Due to the preprocessing steps required, using CURL may not not appropriate. Instead, The API will  be tested by running a python script predict_via_rest_api which generate a request to the predict endpoint.

In [46]:
!python3 predict_via_rest_api.py

## 2.5 Clean-up

Stop and remove the container

In [49]:
!docker stop sertis-detector && docker rm sertis-detector
!docker ps --all

sertis-object-detector
Error: No such container: sertis-object-detector
CONTAINER ID        IMAGE                COMMAND                  CREATED             STATUS                   PORTS               NAMES
50a327d2f825        tensorflow/serving   "/usr/bin/tf_serving…"   28 seconds ago      Up 27 seconds            8500-8501/tcp       serving_base
cbf5a53282d3        jwt-api-test:1.0     "gunicorn -b :8080 m…"   5 weeks ago         Exited (0) 5 weeks ago                       my-app


# 3. Create a docker image to serve the model



## 3.1 Start Tensorflow Serving Server

In [47]:
!docker run -d --name serving_base tensorflow/serving
!docker ps -a

50a327d2f82589895703cd774122bf30df754908bb6412b43de0b1254acb02d8
CONTAINER ID        IMAGE                COMMAND                  CREATED             STATUS                   PORTS                              NAMES
50a327d2f825        tensorflow/serving   "/usr/bin/tf_serving…"   1 second ago        Up Less than a second    8500-8501/tcp                      serving_base
06dfb17894b5        tensorflow/serving   "/usr/bin/tf_serving…"   2 hours ago         Up 2 hours               0.0.0.0:8500-8501->8500-8501/tcp   sertis-object-detector
cbf5a53282d3        jwt-api-test:1.0     "gunicorn -b :8080 m…"   5 weeks ago         Exited (0) 5 weeks ago                                      my-app


## 3.2 Copy the model from local filesystem into container file system

In [51]:
!docker cp serving/sertis-detector serving_base:/models/sertis-detector


## 3.3 Create a docker image with the new change applied

In [53]:
!docker commit --change "ENV MODEL_NAME sertis-detector" serving_base tf-sertis-detector

sha256:b958cf060a5592c166fab0e00b5e4f9f6a201f79a92ad3e8aa91abfa18dbbc60


## 3.4 Stop TF Serving container.

In [81]:
!docker kill serving_base
!docker stop serving_base && docker rm serving_base

Error response from daemon: Cannot kill container: serving_base: No such container: serving_base
Error response from daemon: No such container: serving_base


In [82]:
!docker ps -a

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS                   PORTS                              NAMES
3f6cc2102b4f        b958cf060a55        "/usr/bin/tf_serving…"   22 minutes ago      Up 22 minutes            0.0.0.0:8500-8501->8500-8501/tcp   sertis-detector
cbf5a53282d3        jwt-api-test:1.0    "gunicorn -b :8080 m…"   5 weeks ago         Exited (0) 5 weeks ago                                      my-app


## 3.5 Check if the image is created

In [77]:
!docker images

REPOSITORY                              TAG                 IMAGE ID            CREATED             SIZE
as12production/sertis-object-detector   1.0                 b958cf060a55        24 minutes ago      499MB
tf-sertis-detector                      latest              b958cf060a55        24 minutes ago      499MB
jwt-api-test                            1.0                 f690ec151b72        5 weeks ago         1.04GB
tensorflow/serving                      latest              7c20ddd72597        4 months ago        251MB
python                                  stretch             b9d77e48a75c        8 months ago        940MB
centurylink/dockerfile-from-image       latest              970eaf375dfd        4 years ago         19.2MB


## 3.6 Test the new Image

Start a docker container using the new image. Then run the predict_via_rest_api.py script to test its functionality.

In [62]:
!docker run -it --rm -p 8500:8500 -p 8501:8501 --name sertis-detector --detach \
        -e MODEL_NAME=sertis-detector tf-sertis-detector

3f6cc2102b4fe8a0c4d96f4ed2415560a4f08f7cc18f113af283dfbac5839430


In [65]:
!python3 predict_via_rest_api.py

## 3.7 Clean up

In [83]:
!docker stop sertis-detector && docker rm sertis-detector

sertis-detector
Error: No such container: sertis-detector


# 4. Deploy to Docker hub

Deploy the newly created image to docker hub for distribution.

## 4.1 Tag the image

In [72]:
!docker tag b958cf060a55 as12production/sertis-object-detector:1.0

## 4.2 Log into docker hub

In [75]:
!docker login --username as12production

Password: 



## 4.3 Push the image to docker hub

In [76]:
!docker push as12production/sertis-object-detector:1.0

The push refers to repository [docker.io/as12production/sertis-object-detector]

4b169550: Preparing 
d98b810c: Preparing 
55bd8fcf: Preparing 
61ac0e5e: Preparing 
3374c0b5: Preparing 
fb8f161b: Preparing 
43ea46a8: Preparing 
fcc4a1a8: Preparing 
4b169550: Pushed   248.3MB/248.3MBserving 1.0: digest: sha256:4b6e58f60a825e34a39b968610e9af3deec30f0acdd9c4493f9820a892784ec0 size: 2202


https://hub.docker.com/r/as12production/sertis-object-detector

## 4.4 Test the docker hub image
### 4.4.1 Remove old images

In [84]:
!docker rmi tf-sertis-detector
!docker rmi as12production/sertis-object-detector
!docker rmi tensorflow/serving  
!docker images

Error: No such image: tf-sertis-detector
Untagged: as12production/sertis-object-detector:1.0
Untagged: as12production/sertis-object-detector@sha256:4b6e58f60a825e34a39b968610e9af3deec30f0acdd9c4493f9820a892784ec0
Deleted: sha256:b958cf060a5592c166fab0e00b5e4f9f6a201f79a92ad3e8aa91abfa18dbbc60
Deleted: sha256:3559d4427da077b38cddcf889c1a4c9b385f7f6d58a7daf2d68db863955e7ee9
Deleted: sha256:7c20ddd72597be37ca64e0393fdc219b8906b8709becacf51f746c9f812a8121
Deleted: sha256:a4cc2c00fdca74c89dec852801b1824cb5fd22e90ac97be2e84357ea3145f95b
Deleted: sha256:6decd594d39b31482b1d147650d855358a26fc600dc06a67ca81228bc7feef6c
Deleted: sha256:6e949cb9cd885c847557035e725919b879b201f37df07e2b19fae80a088058a3
Deleted: sha256:2d95a023d1fa3fc0caabcc97ee5dcdb7e75dd79e24567431f8e34047ae660ee7
Deleted: sha256:7c52cdc1e32d67e3d5d9f83c95ebe18a58857e68bb6985b0381ebdcec73ff303
Deleted: sha256:a3c2e83788e20188bb7d720f36ebeef2f111c7b939f1b19aa1b4756791beece0
Deleted: sha256:61199b56f34827cbab596c63fd6e0ac0c448faa7e0

### 4.4.2 Pull the image from docker hub

In [86]:
!docker pull as12production/sertis-object-detector:1.0

1.0: Pulling from as12production/sertis-object-detector

a4a261c9: Pulling fs layer 
20cdee96: Pulling fs layer 
60e1d0de: Pulling fs layer 
7668deea: Pulling fs layer 
b5699598: Pulling fs layer 
8f5dbe31: Pulling fs layer 
011e11a2: Pulling fs layer 
075f0126: Pulling fs layer 
Digest: sha256:4b6e58f60a825e34a39b968610e9af3deec30f0acdd9c4493f9820a892784ec0[9A
Status: Downloaded newer image for as12production/sertis-object-detector:1.0
docker.io/as12production/sertis-object-detector:1.0


### 4.4.3 List all images

In [87]:
!docker images

REPOSITORY                              TAG                 IMAGE ID            CREATED             SIZE
as12production/sertis-object-detector   1.0                 b958cf060a55        29 minutes ago      499MB
jwt-api-test                            1.0                 f690ec151b72        5 weeks ago         1.04GB
python                                  stretch             b9d77e48a75c        8 months ago        940MB


### 4.4.4 Start the downloaded container

In [89]:
!docker run -it --rm -p 8500:8500 -p 8501:8501 --name sertis-detector --detach \
        -e MODEL_NAME=sertis-detector as12production/sertis-object-detector:1.0

ea7018f0e1ff409a97437020fe0e5c903b6cb254ae77ebcd81618d28939dc522


### 4.4.5 Inspect the container

In [91]:
!docker inspect ea7018f0e1ff409a97437020fe0e5c903b6cb254ae77ebcd81618d28939dc522

[
    {
        "Id": "ea7018f0e1ff409a97437020fe0e5c903b6cb254ae77ebcd81618d28939dc522",
        "Created": "2020-05-22T16:35:13.9663642Z",
        "Path": "/usr/bin/tf_serving_entrypoint.sh",
        "Args": [],
        "State": {
            "Status": "running",
            "Running": true,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 6785,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2020-05-22T16:35:14.2965373Z",
            "FinishedAt": "0001-01-01T00:00:00Z"
        },
        "Image": "sha256:b958cf060a5592c166fab0e00b5e4f9f6a201f79a92ad3e8aa91abfa18dbbc60",
        "ResolvConfPath": "/var/lib/docker/containers/ea7018f0e1ff409a97437020fe0e5c903b6cb254ae77ebcd81618d28939dc522/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/ea7018f0e1ff409a97437020fe0e5c903b6cb254ae77ebcd81618d28939dc522/hostname",
        "HostsPath": "/var/lib/docker

### 4.6 Execute the test prediction

In [92]:
!python3 predict_via_rest_api.py

### 4.7 Clean-up

In [94]:
!docker stop sertis-detector  && docker rm sertis-detector
!docker ps -a

sertis-detector
Error: No such container: sertis-detector
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS                   PORTS               NAMES
cbf5a53282d3        jwt-api-test:1.0    "gunicorn -b :8080 m…"   5 weeks ago         Exited (0) 5 weeks ago                       my-app



# 5. Conclusion

In this task, a frozen graph has been successfully converted to a SavedModel and deployed to Tensorflow Serving. A new docker image is created with the model included and is deployed to Docker hub. 


## 5.1 Future Task 

Currently, the model isn't quite user friendly as it requires a specialized image preprocessing before the input can be fed into the prediction service. I believe that the transformation pipeline should be included into the pipeline or create a front-end API that performs this transformation. It is a lot more user friendly if the end API accepts a base64 encoding image and return the predictions. 